In [1]:
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
# -----
from scipy import signal
from skimage.restoration import denoise_tv_chambolle
from skimage.filters import gaussian
import json
import pandas as pd
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
from keras import initializers
from keras import regularizers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, ZeroPadding2D
from keras.layers import BatchNormalization
from keras.utils.np_utils import to_categorical
from keras.layers.advanced_activations import LeakyReLU
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


## Data and parameters

In [2]:
def channels(data):
    rgb_arrays = []
    for i, row in data.iterrows():
        
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2
        
        band_1 = denoise_tv_chambolle(band_1, weight=0.25) # chosen arbitrarily
        band_2 = denoise_tv_chambolle(band_2, weight=0.25)
        band_3 = denoise_tv_chambolle(band_3, weight=0.25)
        
        # gradient magnitude - doesn't seem to work any better than (b1 + b2)/2
#         xder = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
#         yder = np.array([[1,2,1],[0,0,0],[-1,-2,-1]])
#         arrx = signal.convolve2d(band_1, xder, mode='same')
#         arry = signal.convolve2d(band_1, yder, mode='same')
#         band_3 = np.hypot(arrx, arry) 

        rgb = np.dstack((band_1, band_2, band_3))
        rgb_arrays.append(rgb)
        
    rgb_arrays = np.array(rgb_arrays)
    return np.array(rgb_arrays)

In [3]:
X = pd.read_json('train.json')
Y = X['is_iceberg']  # labels

# X = channels(X)
Y = to_categorical(Y)

In [4]:
image_height = 75
image_width = 75
channels_num = 3
number_of_classes = 2

epochs = 30
batch_size = 16

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.15)

X_train = channels(X_train)
X_test = channels(X_test)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(1363, 75, 75, 3)
(241, 75, 75, 3)
(1363, 2)
(241, 2)


## Model

In [11]:
model1 = Sequential()

# 1
model1.add(Conv2D(32, (3, 3), input_shape=(image_width, image_height, channels_num)))
model1.add(LeakyReLU(alpha=.01))
model1.add(MaxPooling2D(pool_size=(2, 2)))

# 2
model1.add(BatchNormalization())
model1.add(Conv2D(64, (3, 3)))
model1.add(LeakyReLU(alpha=.01))
model1.add(MaxPooling2D(pool_size=(2, 2)))

# 3
model1.add(BatchNormalization())
model1.add(Conv2D(128, (3, 3)))
model1.add(LeakyReLU(alpha=.01))
model1.add(MaxPooling2D(pool_size=(2, 2)))

model1.add(Flatten())

# 1
model1.add(BatchNormalization())
model1.add(Dense(256, use_bias=True, bias_initializer='zeros'))
model1.add(LeakyReLU(alpha=.01))
model1.add(Dropout(0.3))

# 2
model1.add(BatchNormalization())
model1.add(Dense(128, use_bias=True, bias_initializer='zeros'))
model1.add(LeakyReLU(alpha=.01))
model1.add(Dropout(0.3))

# 3
model1.add(BatchNormalization())
model1.add(Dense(64, use_bias=True, bias_initializer='zeros'))
model1.add(LeakyReLU(alpha=.01))
model1.add(Dropout(0.3))

model1.add(BatchNormalization())
model1.add(Dense(number_of_classes, activation='softmax'))

In [12]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 73, 73, 32)        896       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 73, 73, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 36, 36, 32)        128       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 34, 34, 64)        0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 17, 17, 64)        0         
__________

In [13]:
model1.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
model1.fit(X_train, Y_train,
              batch_size=batch_size, validation_data=(X_test, Y_test),
              epochs=15,
                shuffle=True)

Train on 1363 samples, validate on 241 samples
Epoch 1/15
1363/1363 [==============================] - 7s 5ms/step - loss: 0.7682 - acc: 0.6273 - val_loss: 1.4396 - val_acc: 0.5187
Epoch 2/15
1363/1363 [==============================] - 6s 4ms/step - loss: 0.6494 - acc: 0.6698 - val_loss: 0.4605 - val_acc: 0.7635
Epoch 3/15
1363/1363 [==============================] - 6s 4ms/step - loss: 0.5607 - acc: 0.7293 - val_loss: 0.5484 - val_acc: 0.6473
Epoch 4/15
1363/1363 [==============================] - 6s 4ms/step - loss: 0.5138 - acc: 0.7454 - val_loss: 0.4067 - val_acc: 0.8091
Epoch 5/15
1363/1363 [==============================] - 6s 4ms/step - loss: 0.4564 - acc: 0.7821 - val_loss: 0.6921 - val_acc: 0.6805
Epoch 6/15
1363/1363 [==============================] - 6s 4ms/step - loss: 0.4107 - acc: 0.7990 - val_loss: 0.3428 - val_acc: 0.8133
Epoch 7/15
1363/1363 [==============================] - 6s 4ms/step - loss: 0.3974 - acc: 0.8188 - val_loss: 0.5762 - val_acc: 0.6929
Epoch 8/15
1363

Reaches ~90% fairly quickly, then becomes a bit unstable :/

## Summary

In [33]:
Y_pred = model1.predict(X_test)
for i in range(len(Y_pred)):
    Y_pred[i, :] = np.round(Y_pred[i, :])


In [34]:
#Precision: how many 1 are true 1?
#Recall: how many 0 are true 0? 
#F1Score: balance between those two

y_comp = confusion_matrix(Y_test[:, 1], Y_pred[:, 1])
precision = y_comp[1, 1] / (y_comp[0, 1] + y_comp[1, 1])
recall = y_comp[1, 1] / (y_comp[1, 0] + y_comp[1, 1])
accuracy = accuracy_score(Y_test[:, 1], Y_pred[:, 1])
f1score = 2 * precision * recall / (precision + recall)
print("AC:%.2f PR:%.2f RE:%.2f F1:%.2f" % (accuracy, precision, recall, f1score))

AC:0.79 PR:0.71 RE:0.94 F1:0.80


In [1]:
filename2 = ("xdddjson")

In [36]:
model1.save(filename)

In [37]:
model_json = model1.to_json()
with open(filename2, "w") as json_file:
    json_file.write(model_json)